In [2]:
import statistics
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
filepath = "/Volumes/mollyT7/2024-04-24/server_pcap/capturequic_s_20240424_1346_5203_pcap.csv"
df = pd.read_csv(filepath, sep=',', encoding='latin-1')


In [4]:
cond_s = df['Source'] == '192.168.1.203'
df_s = df[cond_s]
df_s[:5]
# cond_c = df['ip.src'] == '192.168.1.79'
# df_c  = df[cond_c]
# # frame_type == 3：ACK for STREAM
# cond_ack = df_s['quic.frame_type'].str.contains('3', na=False)
# df_s_ack = df_s[cond_ack]

,No.,Arrival Time,Epoch Time,Time,Source,Destination,Source Port,Destination Port,Protocol,Length,Info,Stream Data,QUIC Short Header
1,2,"Apr 24, 2024 13:47:59.197588000 CST",1.713938e+09,0.001803,192.168.1.203,223.140.16.198,5203,54237,QUIC,1294,"Protected Payload (KP0), PKN: 0, NCI, NCI, NCI",NaN,â\234\223
5,6,"Apr 24, 2024 13:47:59.227128000 CST",1.713938e+09,0.031343,192.168.1.203,223.140.16.198,5203,54237,QUIC,349,"Protected Payload (KP0), PKN: 1, ACK, NT, CRYP...",NaN,â\234\223
7,8,"Apr 24, 2024 13:47:59.230841000 CST",1.713938e+09,0.035056,192.168.1.203,223.140.16.198,5203,54237,HTTP3,1068,"Protected Payload (KP0), PKN: 2, ACK, STREAM(0...",000425d401df5e7666289d0f0dbd5eba00000001bd1c2b...,â\234\223
8,9,"Apr 24, 2024 13:47:59.231478000 CST",1.713938e+09,0.035693,192.168.1.203,223.140.16.198,5203,54237,QUIC,1065,"Protected Payload (KP0), PKN: 3, STREAM(0)",000425d401df5e7666289d0f0dc9826b000000021d385e...,â\234\223
9,10,"Apr 24, 2024 13:47:59.232262000 CST",1.713938e+09,0.036477,192.168.1.203,223.140.16.198,5203,54237,QUIC,1065,"Protected Payload (KP0), PKN: 4, STREAM(0)",000425d401df5e7666289d0f0dd5c00700000003cc90b9...,â\234\223


In [ ]:
# Create an empty list to hold the results
rows = []

delete_row = []

for i in df_s_ack.index:
    idx = df.iloc[i]['quic.ack.largest_acknowledged']
    idx = int(idx)
    filtered_data = df_c[df_c['quic.packet_number'] == str(idx)]
    filtered_data = filtered_data[filtered_data['quic.frame_type'].str.contains('12', na=False)]
    if filtered_data[filtered_data['quic.frame_type'].str.contains('12', na=False)].empty == True:
        df_s_ack = df_s_ack.drop(i)
        print(i, "HI")
        continue
    rows.append(filtered_data)

# Now concatenate the rows to create the resulting DataFrame
df_c_acked_stream = pd.concat(rows) # Transpose the DataFrame if needed

df_c_acked_stream[:5]

In [ ]:
print(df_s_ack.index)
print(len(df_s_ack))

In [ ]:
df['quic.frame_type'].iloc[12]

In [ ]:
if len(df_s_ack) != len(df_c_acked_stream):
    print("length not match", len(df_s_ack), len(df_c_acked_stream))
for i in range(len(df_s_ack)):
    if int(df_c_acked_stream['quic.packet_number'].iloc[i]) != int(df_s_ack['quic.ack.largest_acknowledged'].iloc[i]):
        print("packet not match!", df_c_acked_stream['quic.packet_number'].iloc[i], int(df_s_ack['quic.ack.largest_acknowledged'].iloc[i]))
print("check done!")

## Calculate Round-Trip-Time
1. send time - receive time
2. Show it as an IQR

In [ ]:
rtt_list = []
for i in range(len(df_s_ack)):
    rtt_list.append(1000*((df_s_ack['frame.time_epoch'].iloc[i] - df_c_acked_stream['frame.time_epoch'].iloc[i] - df_s_ack['quic.ack.ack_delay'].iloc[i] / 1000000)))

In [ ]:
stdev = statistics.stdev(rtt_list)
mean = statistics.mean(rtt_list)
median = statistics.median(rtt_list)
print("standard deviation:", stdev)
print("mean:", mean)
print("median:", median)

In [ ]:
q1, q3 = np.percentile(rtt_list, [25, 75])
iqr = q3 - q1

lower_fence = q1 - (1.5*iqr)
higher_fence = q3 + (1.5*iqr)

print(q1, q3, iqr, lower_fence, higher_fence)

In [ ]:
sns.boxplot(rtt_list)
plt.show()

## Re-Transmission

In [ ]:
print(len(df_c_acked_stream))

In [ ]:
x = df_c_acked_stream['quic.stream_data'].iloc[0]
t = int(x[16:24], 16)
usec = int(x[24:32], 16)
# seq = int(x[32:40], 16)
print(t, usec)

In [ ]:
tList = []
usecList = []
t = 0
usec = 0
for i in range(len(df_c_acked_stream)):
    idx = df_c_acked_stream['quic.packet_number'].iloc[i]
    x = df_c_acked_stream['quic.stream_data'].iloc[i]
    t = int(x[16:24], 16)
    usec = int(x[24:32], 16)
    tList.append(t)
    usecList.append(usec)
    print(idx, t, usec)

In [ ]:
df_c_stream = df[df['quic.frame_type'] == '12']

print(len(df_c_stream))
df_c_stream[:5]

In [ ]:
df_c_stream[12:15]
# print(df_c_stream[21])
# print(df_c_stream[22])